In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import geojson
import rasterio
import xarray
import firedrake
import icepack

In [ ]:
outline_filename = icepack.datasets.fetch_outline("pine-island")
with open(outline_filename, "r") as outline_file:
    outline = geojson.load(outline_file)

Figure out which feature and line string corresponds to the ice front.
From this figure it looks like feature 0, line string 0.

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.set_axis_off()

for feature_index, feature in enumerate(outline["features"]):
    for line_index, line_string in enumerate(feature["geometry"]["coordinates"]):
        xs = np.array(line_string)
        label = f"{feature_index} | {line_index}"
        ax.plot(xs[:, 0], xs[:, 1], linewidth=2, label=label)

ax.legend(loc="lower right");

In [ ]:
feature_index = 0
line_string_index = 0
feature = outline["features"][feature_index]
terminus = feature["geometry"]["coordinates"][line_string_index]

Get a bounding box for the domain.

In [ ]:
coords = np.array(list(geojson.utils.coords(outline)))
xs, ys = coords.T
delta = 10e3
xmin, xmax = xs.min() - delta, xs.max() + delta
ymin, ymax = ys.min() - delta, ys.max() + delta

Read the velocity data.
This will be for all of Antarctica, which is huge.

In [ ]:
velocity_filename = icepack.datasets.fetch_measures_antarctica()
velocity_dataset = xarray.open_dataset(velocity_filename)

Subset it to the region around Pine Island.

In [ ]:
pine_island_dataset = velocity_dataset.sel(x=slice(xmin, xmax), y=slice(ymax, ymin))
x, y = pine_island_dataset["x"], pine_island_dataset["y"]
vx, vy = pine_island_dataset["VX"], pine_island_dataset["VY"]

The size will be a few hundred pixels in each direction instead of 12,000.

In [ ]:
vx.shape

Plot the speed so we're sure we're not crazy.

In [ ]:
speed = np.sqrt(vx**2 + vy**2)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")

extent = (xmin, xmax, ymin, ymax)
colors = ax.imshow(speed, extent=extent, cmap="Blues")
fig.colorbar(colors);

Now we'll make a stream plot of the velocity field.
We can use the data from it to pick a flow line.

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.set_axis_off()

X, Y = x.data, y.data[::-1]
VX, VY = vx.data[::-1, :], vy.data[::-1, :]

kw = {
    "start_points": np.array(terminus),
    "integration_direction": "backward",
    "broken_streamlines": False,
    "arrowstyle": "-",
    "density": 3,
}
streamlines = ax.streamplot(X, Y, VX, VY, **kw)

We can pull out the data from this plot.
The streamlines are indexed in increasing order from grid east to west.
I counted manually and the 7th one from the last looks about like the centerline.
Plot it below to be sure.

In [ ]:
segments = streamlines.lines.get_segments()
segment_index = 7
segment = segments[-segment_index]
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.set_axis_off()

colors = ax.imshow(speed, extent=extent, cmap="Blues")
fig.colorbar(colors)
ax.plot(*segment.T, color="tab:red");

Next we want to compute the cumulative length along the streamline.
On average, it looks like most of the points are about 4km apart.
We can make this smaller by adjusting the `"density"` argument to streamplot above; higher density puts in more points.

In [ ]:
dX = np.diff(segment, axis=0)
dl = np.sqrt(np.sum(dX**2, axis=1))
L = np.insert(np.cumsum(dl), 0, 0.0)

In [ ]:
x = xarray.DataArray(segment[:, 0], dims="z")
y = xarray.DataArray(segment[:, 1], dims="z")

u = vx.interp(x=x, y=y)
v = vy.interp(x=x, y=y)

In [ ]:
speed = np.sqrt(u**2 + v**2)

For more sanity checking, plot the speed along the flowline.

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("Distance (meters)")
ax.set_ylabel("Speed (meters/year)")
ax.plot(L, speed);

Now we want to get the surface elevation, bed elevation, and thickness along the flowline.

In [ ]:
bedmachine_filename = icepack.datasets.fetch_bedmachine_antarctica()
bedmachine_dataset = xarray.open_dataset(bedmachine_filename)
thickness = bedmachine_dataset["thickness"]
surface = bedmachine_dataset["surface"]
bed = bedmachine_dataset["bed"]

In [ ]:
h = thickness.interp(x=x, y=y)
s = surface.interp(x=x, y=y)
b = bed.interp(x=x, y=y)

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("Distance (meters)")
ax.set_ylabel("Elevation (meters)")
ax.plot(L, np.zeros_like(L), "--", color="tab:grey")
ax.plot(L, s, color="tab:blue")
ax.plot(L, s - h, color="tab:blue")
ax.plot(L, b, color="tab:brown");